# Portfolio VAR

In [254]:
#Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
from scipy.stats import norm


In [250]:
asset_ril = pd.read_csv('/Users/nesarasr/Documents/Sem6/FRM/Portfolio VAR/RIL.csv')
asset_drl = pd.read_csv('/Users/nesarasr/Documents/Sem6/FRM/Portfolio VAR/DRL.csv')
asset_itc = pd.read_csv('/Users/nesarasr/Documents/Sem6/FRM/Portfolio VAR/ITC.csv')
asset_ntpc = pd.read_csv('/Users/nesarasr/Documents/Sem6/FRM/Portfolio VAR/NTPC.csv')
asset_msl = pd.read_csv('/Users/nesarasr/Documents/Sem6/FRM/Portfolio VAR/MSL.csv')

In [253]:
asset_ril = asset_ril[{'Close Price','Date'}]
asset_drl = asset_drl[{'Close Price','Date'}]
asset_itc = asset_itc[{'Close Price','Date'}]
asset_ntpc = asset_ntpc[{'Close Price','Date'}]
asset_msl = asset_msl[{'Close Price','Date'}]

In [161]:
price_list = [asset_ril,asset_drl,asset_itc,asset_ntpc,asset_msl]

In [162]:
stocks = ['RIL','DRL','ITC','NTPC','MSL']

In [163]:
data = {}
for i in range(len(stocks)):
    data[stocks[i]] = price_list[i] 

In [164]:
returns = {}

for stock in list(data.keys()):
    r = []
    for i in range(0,len(data[stock]['Close Price'])-1):
        r.append(np.log(data[stock]['Close Price'][i]/data[stock]['Close Price'][i+1]))
    returns[stock] = r    

In [165]:
average_returns = {}

for stock in list(returns.keys()):
    
    average_returns[stock] = np.mean(np.array(returns[stock]))
average_returns    



{'RIL': 0.0003025056841690975,
 'DRL': 0.00040887232614948455,
 'ITC': 0.0001227835683981879,
 'NTPC': -0.00028234372282608534,
 'MSL': 0.0007205957710379744}

In [166]:
stddev_returns = {}
for stock in list(returns.keys()):
    
    stddev_returns[stock] = np.std(np.array(returns[stock]))
stddev_returns    



{'RIL': 0.022875187375247543,
 'DRL': 0.016935329570348538,
 'ITC': 0.017842358738408278,
 'NTPC': 0.016936266744191766,
 'MSL': 0.0188494319932806}

In [219]:
number_of_assets = {}
number_of_assets['RIL'] = 500
number_of_assets['DRL'] = 150
number_of_assets['ITC'] = 10000
number_of_assets['NTPC'] = 20000
number_of_assets['MSL'] = 100


In [220]:
amounts_invested = {}

for stock in list(data.keys()):
    amounts_invested[stock] = number_of_assets[stock]*data[stock]['Close Price'][0]
amounts_invested 

{'RIL': 947675.0,
 'DRL': 664290.0,
 'ITC': 2159500.0,
 'NTPC': 1840000.0,
 'MSL': 739995.0}

In [221]:
total_portfolio_value = 0
for stock in list(amounts_invested.keys()):
    total_portfolio_value = total_portfolio_value + amounts_invested[stock]

total_portfolio_value

6351460.0

In [222]:
weight_assets = {}

for stock in list(amounts_invested.keys()):
    
    weight_assets[stock] = amounts_invested[stock]/total_portfolio_value

weight_assets

{'RIL': 0.14920585188287416,
 'DRL': 0.1045885512937183,
 'ITC': 0.34000056679881474,
 'NTPC': 0.28969717198880257,
 'MSL': 0.1165078580357902}

In [223]:
returns_stacked = np.stack((np.transpose(np.array(returns[stocks[0]])),
                            np.transpose(np.array(returns[stocks[1]])),
                            np.transpose(np.array(returns[stocks[2]])),
                            np.transpose(np.array(returns[stocks[3]])),
                            np.transpose(np.array(returns[stocks[4]]))),axis=0)

In [224]:
var_covar_matrix = np.cov(returns_stacked)

In [225]:
asset_values = []
for stock in list(data.keys()):
    asset_values.append(amounts_invested[stock])

In [226]:
asset_values

[947675.0, 664290.0, 2159500.0, 1840000.0, 739995.0]

In [227]:
portfolio_variance = (np.array(asset_values) @ var_covar_matrix @ np.transpose(np.array(asset_values)))
portfolio_stddev = np.sqrt(portfolio_variance)

In [228]:
portfolio_variance_returns = portfolio_variance/(total_portfolio_value**2)
portfolio_stddev_returns = np.sqrt(portfolio_variance_returns)

In [229]:
# at confidence level = 99%
Z = norm.ppf(0.99)

In [230]:
Z

2.3263478740408408

### portfolio var : diversified

In [231]:
pvar_diversified = Z*portfolio_stddev

In [232]:
pvar_diversified

170593.45851968418

### portfolio var : undiversified

In [233]:
pvar_undiversified = 0
for stock in stocks:
    pvar_undiversified = pvar_undiversified+np.abs(Z*amounts_invested[stock]*stddev_returns[stock])
    

In [234]:
pvar_undiversified

271182.3791419533

### marginal var

In [235]:
marginal_var = Z* (var_covar_matrix @ np.transpose(np.array(asset_values)) ) / (total_portfolio_value*portfolio_stddev_returns)

In [236]:
marginal_var

array([0.03018459, 0.01424054, 0.03017353, 0.02693201, 0.02407288])

### incremental var

In [237]:
change_in_no_of_assets = [0,1,0,-1000,0]
change_in_assets = []

for i in range(len(stocks)):
    change_in_assets.append(change_in_no_of_assets[i]*data[stocks[i]]['Close Price'][0])

In [238]:
ivar = marginal_var @ np.transpose(np.array(change_in_assets))

In [239]:
ivar

-2414.678873979051

In [240]:
change_in_assets

[0.0, 4428.6, 0.0, -92000.0, 0.0]

In [241]:
new_var = pvar + ivar

In [242]:
new_var

168178.77964570513

### Add new asset

In [243]:
asset_kmb = pd.read_csv('/Users/nesarasr/Documents/Sem6/FRM/Portfolio VAR/KMB.csv')

In [244]:
asset_kmb = asset_kmb[{'Close Price','Date'}]

In [245]:
return_new = []
for i in range(0,len(data[stock]['Close Price'])-1):
    return_new.append(np.log(asset_kmb['Close Price'][i]/asset_kmb['Close Price'][i+1]))
  

2478

In [458]:
#Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
from scipy.stats import norm


Money is mainly invested in index instruments, allotting 35% to broad market cap, 25% to thematic index, 20% to strategy, 10% to sector & industry, and 10% to sustainability.

## Question 3

In [459]:
returns = pd.read_csv('/Users/nesarasr/Documents/Sem6/FRM/ClassTest2/returns.csv')

In [460]:
returns = returns[:-1]
returns['Sensex'] = np.float64(returns['Sensex'])

In [461]:
portfolio = returns.columns[1:]

In [462]:
portfolio

Index(['MarketCap', 'Strategy', 'Sustainability', 'Sector & Industry',
       'Thematic', 'Nestle', 'Sensex'],
      dtype='object')

In [463]:
old_portfolio = portfolio[0:5]
new_portfolio = portfolio[0:6]
market_returns = portfolio[6]

In [464]:
amounts_invested_old_portfolio = [35,20,10,10,25]  ## ['MarketCap', 'Strategy', 'Sustainability', 'Sector & Industry','Thematic']
amounts_invested_new_portfolio = [35,20,10,10,25,25] ## ['MarketCap', 'Strategy', 'Sustainability', 'Sector & Industry','Thematic','Nestle']

_amounts invested in crores*_

In [465]:
total_portfolio_old = sum(amounts_invested_old_portfolio)

### Beta Model

#### computing beta values of components of old portfolio

In [466]:
beta = []
for asset in (old_portfolio):
    beta.append(returns[market_returns].cov(returns[asset])/returns[asset].var())

In [467]:
beta

[0.9999999999999992,
 0.773036703992015,
 0.9943407673625877,
 0.5485864061288108,
 0.7122367511780304]

In [468]:
weights_old = np.array(amounts_invested_old_portfolio)/(np.sum(np.array(amounts_invested_old_portfolio)))

In [469]:
beta_portfolio = np.array(weights_old) @ np.array(beta).transpose()

In [470]:
beta_portfolio

0.8369592459420501

In [471]:
market_variance = returns[market_returns].var()

In [472]:
old_portfolio_variance_beta_model = beta_portfolio*market_variance

In [473]:
print("Beta Model Variance of returns of OLD Portfolio:", old_portfolio_variance_beta_model)
print("Beta Model Variance of OLD Portfolio:", old_portfolio_variance_beta_model*(total_portfolio_old**2)," crores^2")

Beta Model Variance of returns of OLD Portfolio: 0.00014781757482611532
Beta Model Variance of OLD Portfolio: 1.4781757482611533  crores^2


### Historical variance-covariance model

In [474]:
list(old_portfolio)

['MarketCap', 'Strategy', 'Sustainability', 'Sector & Industry', 'Thematic']

In [475]:
returns_stacked = np.stack((np.transpose(np.array(returns[old_portfolio[0]])),
                            np.transpose(np.array(returns[old_portfolio[1]])),
                            np.transpose(np.array(returns[old_portfolio[2]])),
                            np.transpose(np.array(returns[old_portfolio[3]])),
                            np.transpose(np.array(returns[old_portfolio[4]]))),axis=0)

In [476]:
var_covar_matrix = np.cov(returns_stacked)

In [477]:
portfolio_variance = (np.array(amounts_invested_old_portfolio) @ var_covar_matrix @ np.transpose(np.array(amounts_invested_old_portfolio)))

In [478]:
portfolio_variance_returns = portfolio_variance/(total_portfolio_old**2)

In [479]:
print("Historical Variance-Covariance Model variance  of returns of OLD Portfolio:", portfolio_variance_returns)
print("Historical Variance-Covariance Model variance of OLD Portfolio:", portfolio_variance," crores^2")

Historical Variance-Covariance Model variance  of returns of OLD Portfolio: 0.00015982166767586833
Historical Variance-Covariance Model variance of OLD Portfolio: 1.5982166767586832  crores^2


## Question 4

### Pre-donation VaR

In [480]:
list(old_portfolio)

['MarketCap', 'Strategy', 'Sustainability', 'Sector & Industry', 'Thematic']

In [481]:
returns_stacked = np.stack((np.transpose(np.array(returns[old_portfolio[0]])),
                            np.transpose(np.array(returns[old_portfolio[1]])),
                            np.transpose(np.array(returns[old_portfolio[2]])),
                            np.transpose(np.array(returns[old_portfolio[3]])),
                            np.transpose(np.array(returns[old_portfolio[4]]))),axis=0)

In [482]:
var_covar_matrix = np.cov(returns_stacked)

In [483]:
portfolio_variance = (np.array(amounts_invested_old_portfolio) @ var_covar_matrix @ np.transpose(np.array(amounts_invested_old_portfolio)))

In [484]:
portfolio_stddev = np.sqrt(portfolio_variance)

In [485]:
portfolio_variance_returns = portfolio_variance/(total_portfolio_old**2)

In [486]:
# at confidence level = 99%
Z = norm.ppf(0.99)

In [487]:
old_portfolio_var = Z*portfolio_stddev

In [488]:
print("99% VAR of Pre-Donation Portfolio:", old_portfolio_var,"crores")


99% VAR of Pre-Donation Portfolio: 2.9409828174554997 crores


### Post-donation VaR

In [489]:
list(new_portfolio)

['MarketCap',
 'Strategy',
 'Sustainability',
 'Sector & Industry',
 'Thematic',
 'Nestle']

In [490]:
total_portfolio_new = sum(amounts_invested_new_portfolio)

In [491]:
returns_stacked = np.stack((np.transpose(np.array(returns[new_portfolio[0]])),
                            np.transpose(np.array(returns[new_portfolio[1]])),
                            np.transpose(np.array(returns[new_portfolio[2]])),
                            np.transpose(np.array(returns[new_portfolio[3]])),
                            np.transpose(np.array(returns[new_portfolio[4]])),
                            np.transpose(np.array(returns[new_portfolio[5]]))),axis=0)

In [492]:
var_covar_matrix = np.cov(returns_stacked)

In [493]:
portfolio_variance = (np.array(amounts_invested_new_portfolio) @ var_covar_matrix @ np.transpose(np.array(amounts_invested_new_portfolio)))

In [494]:
portfolio_stddev = np.sqrt(portfolio_variance)

In [495]:
portfolio_variance_returns = portfolio_variance/(total_portfolio_new**2)
portfolio_stddev_returns = np.sqrt(portfolio_variance_returns)

In [496]:
# at confidence level = 99%
Z = norm.ppf(0.99)

In [497]:
new_portfolio_var = Z*portfolio_stddev

In [498]:
print("99% VAR of Post-Donation Portfolio:", new_portfolio_var,"crores")


99% VAR of Post-Donation Portfolio: 3.110867895605865 crores


## Question 5

### Component - VAR = A.t * Marginal VAR

#### Marginal VAR of new portfolio

In [499]:
# at confidence level = 99%
Z = norm.ppf(0.99)

marginal_var = Z* (var_covar_matrix @ np.transpose(np.array(amounts_invested_new_portfolio)) ) / (total_portfolio_new*portfolio_stddev_returns)

In [500]:
marginal_var

array([0.02787933, 0.0291532 , 0.02805801, 0.02061639, 0.03010118,
       0.01251015])

In [501]:
risk_components = []
for i in range(len(amounts_invested_new_portfolio)):
    
    print("Risk contribution of",new_portfolio[i]," in the post donation portfolio is",marginal_var[i]*amounts_invested_new_portfolio[i],"crores")
    risk_components.append(marginal_var[i]*amounts_invested_new_portfolio[i])
    

Risk contribution of MarketCap  in the post donation portfolio is 0.9757765698469184 crores
Risk contribution of Strategy  in the post donation portfolio is 0.5830639512023121 crores
Risk contribution of Sustainability  in the post donation portfolio is 0.2805800884630307 crores
Risk contribution of Sector & Industry  in the post donation portfolio is 0.20616389658255702 crores
Risk contribution of Thematic  in the post donation portfolio is 0.7525295874324434 crores
Risk contribution of Nestle  in the post donation portfolio is 0.3127538020786034 crores


## Question 6

### On selling 5% of X = Nestle 
#### change in VAR (Incremental VAR) = change_in_portfolio.T X marginal_var

In [502]:
fraction_change_in_portfolio = [0,0,0,0,0,-0.05]

In [503]:
change_in_portfolio = []
for i in range(len(amounts_invested_new_portfolio)):
    change_in_portfolio.append(fraction_change_in_portfolio[i]*amounts_invested_new_portfolio[i])

In [504]:
change_in_portfolio

[0, 0, 0, 0, 0, -1.25]

In [505]:
ivar = np.array(change_in_portfolio) @ np.transpose(np.array(marginal_var))

In [506]:
new_var = new_portfolio_var+ivar

In [507]:
print("Expected change in VaR of the portfolio from selling 5% of X :", ivar,"crores")
print("New VaR of the portfolio resulting from selling 5% of X :", new_var,"crores")

Expected change in VaR of the portfolio from selling 5% of X : -0.015637690103930173 crores
New VaR of the portfolio resulting from selling 5% of X : 3.0952302055019345 crores


## Question 7 

### change in portfolio position so that the post-donation portfolio VaR equals VaR of the pre-donation portfolio

In [520]:
print("lowest risk constituent of new portfolio :",new_portfolio[np.argmin(marginal_var)])

lowest risk constituent of new portfolio : Nestle


'X' is the lowest risk constituent. Therefore I have computed the amount of highest risk constituent asset that needs to be replaced with 'X' to bring the VAR down to that of pre-donation portfolio

In [522]:
print("highest risk constituent of new portfolio :",new_portfolio[np.argmax(marginal_var)])

highest risk constituent of new portfolio : Thematic


#### Explanation

let 'p' be the total value of 'X' that needs to be replaced with lowest risk constituent.

We have p * (marginal_var("X")- marginal_var("lowest risk constituent")) = new_portfolio_var - old_portfolio_var 

Therefore, p = ( new_portfolio_var - old_portfolio_var ) / (marginal_var("X")- marginal_var("lowest risk constituent"))

In [523]:
p = ( new_portfolio_var - old_portfolio_var ) / (marginal_var[np.argmax(marginal_var)]- marginal_var[np.argmin(risk_components)])

In [524]:
p

17.911309516404163

In [527]:
print("Substituting",p,"crores of Thematic with Nestle in the portfolio will bring down the VAR to that of old(pre-donation) portfolio")

Substituting 17.911309516404163 crores of Thematic with Nestle in the portfolio will bring down the VAR to that of old(pre-donation) portfolio
